# Script to plot UVA lamp spectra and cross section with photolysis frequency calculations

## load python packages

In [18]:
## the script is located at 5505/People/Fengxia_Bao/5505_fengxia/5505_python_scripts/photolysis_rate_cross_section_final.ipynb
## the cross section data is located at 5505/People/Fengxia_Bao/5505_fengxia/5505_python_scripts/actinic_flux/ or on the url/references provided in the script

In [2]:
import numpy as np
import pandas as pd
from pandas import Series
import os
from pandas import DataFrame
import pickle 
from datetime import datetime
import statistics
import csv
import itertools
import math
from math import cos, exp, pi
import urllib3
import io
import xlsxwriter
from openpyxl import load_workbook


## import data
## plot lamps actinic flux spectra

In [3]:
filepath="ExampleLightFlux_PSI_chamber.xlsx"  
psi_chamber_df=pd.read_excel(filepath,header=None).astype(float)
psi_chamber_df.rename(columns={psi_chamber_df.columns[0]: 'wavelength'}, inplace=True)
psi_chamber_df.rename(columns={psi_chamber_df.columns[1]: 'flux'}, inplace=True)

In [5]:
filepath="actinic_flux/Yulia/Lamps_small.DAT"  
small_lamps_df=pd.read_csv(filepath,sep='/t',encoding='unicode_escape')
small_lamps_df=small_lamps_df.iloc[32:,0].str.split(",", expand = True).astype(float)
small_lamps_df.rename(columns={small_lamps_df.columns[0]: 'wavelength'}, inplace=True)
# pd.read_csv(filepath,sep='/t',encoding='unicode_escape').iloc[10:31,0]

/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_30432/518281335.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  small_lamps_df=pd.read_csv(filepath,sep='/t',encoding='unicode_escape')


In [6]:
actinic=pd.DataFrame()
actinic['wavelength']=small_lamps_df.iloc[:,0]
actinic['Yulia UVA 7 lamps old box']=small_lamps_df.iloc[:,1]
actinic['Yulia UVA 7 lamps new box']=small_lamps_df.iloc[:,9]
actinic['Yulia VIS 7 lamps old box']=small_lamps_df.iloc[:,5]
actinic['Yulia VIS 7 lamps new box']=small_lamps_df.iloc[:,13]

excel_path = 'photolysis_calculation_bao.xlsx'
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
# actinic.to_excel(excel_path,sheet_name='actinic')
actinic.to_excel(writer, sheet_name = 'actinic')
writer.close()

## import data of cross section and plot

##  nitrate
http://doi.wiley.com/10.1562/0031-8655%282000%290710431DAIOUS2.0.CO2

In [7]:
cross_section_nitrate = pd.read_csv('actinic_flux/nitrate_cross_section_Jankowski2007.csv').astype(float) 
wavelength = cross_section_nitrate.iloc[:350-290+1,0]
molar_absorption_coefficients= cross_section_nitrate.iloc[:350-290+1,1]   ## 10^3 cm2 mol^-1
Na = 6.022140957e23

new_cross_section_nitrate=pd.DataFrame()
new_cross_section_nitrate['wavelength']=wavelength
new_cross_section_nitrate['molar_absorption_coefficients']=molar_absorption_coefficients.values
molecular_cross_section = molar_absorption_coefficients* math.log(10) * 1000 / Na   #cm2 molecule-1   math.log(10, math.e) = math.log(10)  convert Molar_absorption_coefficients to molecular cross section 
new_cross_section_nitrate['molecular_cross_section']=molecular_cross_section
new_cross_section_nitrate['quantumn_yield']=0.01    #assuming to be 0.01  however, should change with wavelength
new_cross_section_nitrate=new_cross_section_nitrate.set_index('wavelength')


In [8]:
cross_section_nitrate = pd.read_csv('actinic_flux/nitrate_epsilon_v1.txt',sep='\t').astype(float)
wavelength = cross_section_nitrate.iloc[:,0]
molar_absorption_coefficients= cross_section_nitrate.iloc[:,1]   ## 10^3 cm2 mol^-1
Na = 6.022140957e23

new_cross_section_nitrate_extended=pd.DataFrame()
new_cross_section_nitrate_extended['wavelength']=wavelength
new_cross_section_nitrate_extended['molar_absorption_coefficients']=molar_absorption_coefficients.values
molecular_cross_section = molar_absorption_coefficients* math.log(10) * 1000 / Na   #cm2 molecule-1   math.log(10, math.e) = math.log(10)  convert Molar_absorption_coefficients to molecular cross section 
new_cross_section_nitrate_extended['molecular_cross_section']=molecular_cross_section
new_cross_section_nitrate_extended['quantumn_yield']=0.01    #assuming to be 0.01  however, should change with wavelength
new_cross_section_nitrate_extended=new_cross_section_nitrate_extended.set_index('wavelength')


In [9]:
Episilon_DHB = pd.read_csv('actinic_flux/DHB_UV_VIS_NIST.txt',sep='\t')
wavelength = Episilon_DHB['wavelength, Logarithm_epsilon '].str.split(',', expand=True).iloc[:,0]
lograithm_epsilon= Episilon_DHB['wavelength, Logarithm_epsilon '].str.split(',', expand=True).iloc[:,1]   ## 
# Na = 6.022140957e23

episilon_DHB_extended=pd.DataFrame()
episilon_DHB_extended['wavelength']=wavelength
episilon_DHB_extended['lograithm_epsilon']=lograithm_epsilon.values
episilon_DHB_extended['lograithm_epsilon']=10**(episilon_DHB_extended['lograithm_epsilon'].astype(float))
molecular_cross_section = episilon_DHB_extended['lograithm_epsilon']* math.log(10) * 1000 / Na   #cm2 molecule-1   math.log(10, math.e) = math.log(10)  convert Molar_absorption_coefficients to molecular cross section 
episilon_DHB_extended['molecular_cross_section']=molecular_cross_section
# episilon_DHB_extended['quantumn_yield']=0.01    #assuming to be 0.01  however, should change with wavelength
episilon_DHB_extended=episilon_DHB_extended.set_index('wavelength')
episilon_DHB_extended

import plotly.graph_objects as go

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= episilon_DHB_extended.index,y=episilon_DHB_extended['molecular_cross_section']*1e17,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=4,
                           name="σ(NO3-)"))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  dtick = 50,
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=6,ticks='inside'),
    yaxis=dict( title='Molecular cross section (10<sup>-17</sup> cm<sup>2</sup>)', zeroline=False,
              #  range=[0,4.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=6,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.write_html('cross_section_actinic_flux_202309051909.html')
fig_1.write_image('DHB_absorption.png')



In [10]:
Absorption_DHB = pd.read_csv('actinic_flux/DHB_absorption_jiang_2021.csv',header=None)
# DOI: 10.1021/acs.est.0c07581 
wavelength = Absorption_DHB.iloc[:,0]
absorption= Absorption_DHB.iloc[:,1]  #unknown units  DOI: 10.1021/acs.est.0c07581  
import plotly.graph_objects as go

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= wavelength,y=absorption*1e4,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=4,
                           name="σ(NO3-)"))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  dtick = 50,
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=6,ticks='inside'),
    yaxis=dict( title='Absorption (a.u.)', zeroline=False,
              #  range=[0,4.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=6,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.write_html('cross_section_actinic_flux_202309051909.html')
fig_1.write_image('DHB_absorption.png')

In [11]:
# new_cross_section_nitrate['wavelength']=wavelength
# new_cross_section_nitrate['molar_absorption_coefficients']=molar_absorption_coefficients.values
fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= new_cross_section_nitrate.index,y= new_cross_section_nitrate['molecular_cross_section']*1e20,yaxis = "y2",mode='markers+lines',marker_size=3,line_width=4,
                           name="NO<sub>3</sub>-<sup></sup>"))
fig_1.add_trace(go.Scatter(x= wavelength,y=absorption*1e4,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=4,name="DHB"))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  dtick = 50,
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='Absorption (a.u.)', zeroline=False,
              #  range=[0,4.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.65,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.write_html('cross_section_actinic_flux_202309051909.html')
fig_1.write_image('DHB_nitrate_absorption.png')

### nitrite
 http://doi.wiley.com/10.1562/0031-8655%282000%290710431DAIOUS2.0.CO2

In [12]:
cross_section_nitrite = pd.read_csv('actinic_flux/nitrate_cross_section_Jankowski2007.csv').astype(float) 
wavelength = cross_section_nitrite.iloc[:,0]
molar_absorption_coefficients= cross_section_nitrite.iloc[:,2]   ## 10^3 cm2 mol^-1

new_cross_section_nitrite=pd.DataFrame()
new_cross_section_nitrite['wavelength']=wavelength
new_cross_section_nitrite['molar_absorption_coefficients']=molar_absorption_coefficients.values
molecular_cross_section = molar_absorption_coefficients* math.log(10) * 1000 / Na   #cm2 molecule-1   math.log(10, math.e) = math.log(10)  convert Molar_absorption_coefficients to molecular cross section 
new_cross_section_nitrite['molecular_cross_section']=molecular_cross_section
new_cross_section_nitrite['quantumn_yield']=1    #assuming to be constnat however, should change with wavelength
new_cross_section_nitrite=new_cross_section_nitrite.set_index('wavelength')


I2

In [14]:
# Creating a PoolManager instance for sending requests.
http = urllib3.PoolManager()
# Sending a GET request and getting back response as HTTPResponse object.
resp = http.request("GET", "https://uv-vis-spectral-atlas-mainz.org/uvvis_data/cross_sections/Halogens+mixed%20halogens/I2_JPL-2010(2011)_295K_185-700nm(rec).txt")
df = pd.read_csv(io.BytesIO(bytearray(bytes(resp.data))), sep='\t',header=None)
new_cross_section_I2=pd.DataFrame()
new_cross_section_I2['wavelength']=df.iloc[19:-10,0].astype(float)
new_cross_section_I2['molecular_cross_section']=df.iloc[19:-10,1].astype(float)
new_cross_section_I2=new_cross_section_I2.set_index('wavelength')
new_cross_section_I2['quantumn_yield']=1    #assuming to be constnat however, should change with wavelength


ClNO

In [15]:

# Creating a PoolManager instance for sending requests.
http = urllib3.PoolManager()
# Sending a GET request and getting back response as HTTPResponse object.
resp = http.request("GET", "https://uv-vis-spectral-atlas-mainz.org/uvvis_data/cross_sections/Halogenated%20N-compounds(inorg)/ClNO_JPL-2010(2011)_298K_190-500nm(rec).txt")
df = pd.read_csv(io.BytesIO(bytearray(bytes(resp.data))), sep='\t',header=None)
new_cross_section_clno=pd.DataFrame()
new_cross_section_clno['wavelength']=df.iloc[45:,0].astype(float)
new_cross_section_clno['molecular_cross_section']=df.iloc[45:,1].astype(float)
new_cross_section_clno=new_cross_section_clno.set_index('wavelength')
new_cross_section_clno['quantumn_yield']=1    #assuming to be constnat however, should change with wavelength


## NO2

In [16]:

# Creating a PoolManager instance for sending requests.
http = urllib3.PoolManager()
# Sending a GET request and getting back response as HTTPResponse object.
resp = http.request("GET", "https://uv-vis-spectral-atlas-mainz.org/uvvis_data/cross_sections/Nitrogen%20oxides/NO2_Davidson(1988)_298K_263-649nm.txt")
df = pd.read_csv(io.BytesIO(bytearray(bytes(resp.data))), sep='\t',header=None)
new_cross_section_NO2=pd.DataFrame()
new_cross_section_NO2['wavelength']=df.iloc[31:,0]
new_cross_section_NO2['molecular_cross_section']=df.iloc[31:,1]
new_cross_section_NO2['quantumn_yield']=1    #assuming to be constnat however, should change with wavelength
new_cross_section_NO2=new_cross_section_NO2.set_index('wavelength')


## HONO

In [17]:

# Creating a PoolManager instance for sending requests.
http = urllib3.PoolManager()
# Sending a GET request and getting back response as HTTPResponse object.
resp = http.request("GET", "https://uv-vis-spectral-atlas-mainz.org/uvvis_data/cross_sections/Nitrogen+compounds(N,H,O)/HONO_JPL-2010(2011)_298K_184-396nm(rec).txt")
df = pd.read_csv(io.BytesIO(bytearray(bytes(resp.data))), sep='\t',header=None)
new_cross_section_HONO=pd.DataFrame()
new_cross_section_HONO['wavelength']=df.iloc[46:,0]
new_cross_section_HONO['molecular_cross_section']=df.iloc[46:,1]
new_cross_section_HONO['quantumn_yield']=1    #assuming to be constnat however, should change with wavelength
new_cross_section_HONO=new_cross_section_HONO.set_index('wavelength')


In [19]:
import plotly.graph_objects as go

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= new_cross_section_nitrate.index,y=new_cross_section_nitrate['molecular_cross_section']*10,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO3-)*10"))
fig_1.add_trace(go.Scatter(x= new_cross_section_nitrite.index,y=new_cross_section_nitrite['molecular_cross_section']*10,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO2-)*10"))
fig_1.add_trace(go.Scatter(x= new_cross_section_I2.index,y=new_cross_section_I2['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(I2)"))
fig_1.add_trace(go.Scatter(x= new_cross_section_clno.index,y=new_cross_section_clno['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(ClNO)"))
fig_1.add_trace(go.Scatter(x= new_cross_section_NO2.index,y=new_cross_section_NO2['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO2)"))
fig_1.add_trace(go.Scatter(x= new_cross_section_HONO.index,y=new_cross_section_HONO['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(HONO)"))


fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,1], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[1]))
fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,2], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[2]))
fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,3], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[3]))
fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,4], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[4]))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1600, height=900,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=1,ticks='inside'),
    yaxis=dict( title='Molecular cross section (cm<sup>2</sup>)', titlefont=dict(color="black",size=25),tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=1,ticks='inside'),
    yaxis2=dict(title='Photon flux (cm<sup>-2</sup>  s<sup>-1</sup> nm<sup>-1</sup>)',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=1,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.1,  font=dict( size=20,color="black")))
config = {'displayModeBar': True}

fig_1.show(config=config)
fig_1.write_html('cross_section_actinic_flux_202309051909.html')


In [20]:
import plotly.graph_objects as go

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= new_cross_section_nitrate.index,y=new_cross_section_nitrate['molecular_cross_section']*1e20,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=4,
                           name="σ(NO3-)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_nitrite.index,y=new_cross_section_nitrite['molecular_cross_section']*10,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO2-)*10"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_I2.index,y=new_cross_section_I2['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(I2)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_clno.index,y=new_cross_section_clno['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(ClNO)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_NO2.index,y=new_cross_section_NO2['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO2)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_HONO.index,y=new_cross_section_HONO['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(HONO)"))


# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,1], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[1]))
# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,2], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[2]))
# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,3], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[3]))
# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,4], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[4]))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)
# fig_1.update_layout(
#   plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=500,
#     xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=1,ticks='inside'),
#     yaxis=dict( title='Molecular cross section (cm<sup>2</sup>)', titlefont=dict(color="black",size=25),tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=1,ticks='inside'),
#     yaxis2=dict(title='Photon flux (cm<sup>-2</sup>  s<sup>-1</sup> nm<sup>-1</sup>)',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=1,showticklabels = True) )
# fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.1,  font=dict( size=20,color="black")))
# config = {'displayModeBar': True}



fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=6,ticks='inside'),
    yaxis=dict( title='Molecular cross section (10<sup>-20</sup> cm<sup>2</sup>)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=6,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.show(config=config)
# fig_1.write_html('cross_section_actinic_flux_202309051909.html')
fig_1.write_image('nitrate_absorption.png')



In [25]:
solar_actinic_flux=pd.read_excel('actinic_flux/solar_actinic_flux.xlsx',header=None)
new_solar_actinic_flux=pd.DataFrame()

new_solar_actinic_flux['wavelength']=solar_actinic_flux.iloc[:,1].values
new_solar_actinic_flux['flux_1']=solar_actinic_flux.iloc[:,2].values
new_solar_actinic_flux['flux_2']=solar_actinic_flux.iloc[:,3].values
new_solar_actinic_flux['flux_3']=solar_actinic_flux.iloc[:,4].values

new_solar_actinic_flux['flux_4']=solar_actinic_flux.iloc[:,5].values
new_solar_actinic_flux['flux_5']=solar_actinic_flux.iloc[:,6].values

new_solar_actinic_flux=new_solar_actinic_flux

import plotly.graph_objects as go

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= new_cross_section_nitrate.index,y=new_cross_section_nitrate['molecular_cross_section']*1e20,yaxis = "y1",
                           mode='markers+lines',showlegend=True,marker_size=5,line_width=5,name="Cross section (NO<sub>3</sub><sup>-</sup>)"))

# 
# fig_1.add_trace(go.Scatter(x= new_solar_actinic_flux.iloc[:,0], y=new_solar_actinic_flux.iloc[:,1], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Solar actinic flux'))
# fig_1.add_trace(go.Scatter(x= new_solar_actinic_flux.iloc[:,0], y=new_solar_actinic_flux.iloc[:,2], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),showlegend=False,marker_size=3,line_width=5,name='Flux '+actinic.columns[1]))
# fig_1.add_trace(go.Scatter(x= new_solar_actinic_flux.iloc[:,0], y=new_solar_actinic_flux.iloc[:,3], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),showlegend=False,marker_size=3,line_width=5,name='Flux '+actinic.columns[1]))

# fig_1.add_trace(go.Scatter(x= new_solar_actinic_flux.iloc[:,0], y=new_solar_actinic_flux.iloc[:,4], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),showlegend=False,marker_size=3,line_width=5,name='Flux '+actinic.columns[1]))


fig_1.add_trace(go.Scatter(x= new_solar_actinic_flux.iloc[:,0], y=new_solar_actinic_flux.iloc[:,5]/1e14, yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),showlegend=True,marker_size=3,line_width=5,name='Solar actinic flux'))
# fig_1.add_trace(go.Scatter(x= new_solar_actinic_flux.iloc[:,0], y=new_solar_actinic_flux.iloc[:,1], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[1]))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=30),showline=True, mirror= True,tickfont=dict(color='black',size=30 ),linecolor="black", linewidth=3,ticks='inside'),
    yaxis=dict( title='Molecular cross section (10<sup>20</sup> cm<sup>2</sup>)', 
              #  range=[0,0.7],
               titlefont=dict(color="blue",size=30),tickfont=dict(color='blue',size=30 ),linecolor="blue", linewidth=3,ticks='inside'),
    yaxis2=dict(title='Solar actinic flux (10<sup>14</sup> photons cm<sup>-2</sup>  s<sup>-1</sup> nm<sup>-1</sup>)',range=[0,7],titlefont=dict(color="red",size=30),tickfont=dict(color="red" ,size=30),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="red", linewidth=3,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.3 ,xanchor="left",x=0.5,  font=dict( size=25,color="black")))
config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.write_html('cross_section_actinic_flux_202309051909.html')

fig_1.write_image("fig2.svg")



In [26]:
import plotly.graph_objects as go

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= new_cross_section_nitrate.index,y=new_cross_section_nitrate['molecular_cross_section']*1e20,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=3,name="σ(NO3-)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_nitrite.index,y=new_cross_section_nitrite['molecular_cross_section']*10,yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO2-)*10"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_I2.index,y=new_cross_section_I2['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(I2)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_clno.index,y=new_cross_section_clno['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(ClNO)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_NO2.index,y=new_cross_section_NO2['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(NO2)"))
# fig_1.add_trace(go.Scatter(x= new_cross_section_HONO.index,y=new_cross_section_HONO['molecular_cross_section'],yaxis = "y1",mode='markers+lines',marker_size=3,line_width=2,name="σ(HONO)"))


fig_1.add_trace(go.Scatter(x= psi_chamber_df.iloc[:,0], y=psi_chamber_df.iloc[:,1]*1e-13, yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Actinic flux'))
# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,2], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[2]))
# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,3], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[3]))
# fig_1.add_trace(go.Scatter(x= actinic.iloc[:,0], y=actinic.iloc[:,4], yaxis = "y2",mode='markers+lines', line=dict(dash='dot'),marker_size=3,line_width=5,name='Flux '+actinic.columns[4]))

fig_1.update_layout(title='',titlefont=dict(color="black",size=25))
fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1600*0.6, height=900*0.6,
    xaxis=dict( title="Wavelength (nm)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=1,ticks='inside'),
    yaxis=dict( title='Molecular cross section (10<sup>20</sup> cm<sup>2</sup>)', titlefont=dict(color="black",size=25),tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=1,ticks='inside'),
    yaxis2=dict(title='Photon flux (10<sup>13</sup> photons cm<sup>-2</sup>  s<sup>-1</sup> nm<sup>-1</sup>)',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=1,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.5,  font=dict( size=20,color="black")))
config = {'displayModeBar': True}

fig_1.show(config=config)


In [27]:
nitrate_df_chamber=(pd.merge(psi_chamber_df, new_cross_section_nitrate, on='wavelength'))
delta_lamda= nitrate_df_chamber.wavelength[1]-nitrate_df_chamber.wavelength[0]
nitrate_1=((nitrate_df_chamber.iloc[:,1]*nitrate_df_chamber['molecular_cross_section']*nitrate_df_chamber['quantumn_yield']).sum(axis=0)*delta_lamda)
# summary_df['nitrate']=[nitrate_1, nitrate_2, nitrate_3,nitrate_4]
nitrate_1

nitrate_df_chamber.to_excel('chamber_JNO3-.xlsx')

### calculate photolysis rate constant

In [ ]:
summary_df=pd.DataFrame()
summary_df['Photolysis rate constant']=actinic.columns[1:5]
summary_df.set_index('Photolysis rate constant')

""
Photolysis rate constant
Yulia UVA 7 lamps old box
Yulia UVA 7 lamps new box
Yulia VIS 7 lamps old box
Yulia VIS 7 lamps new box


In [ ]:
nitrate_df=(pd.merge(actinic, new_cross_section_nitrate, on='wavelength'))
delta_lamda= nitrate_df.wavelength[1]-nitrate_df.wavelength[0]
nitrate_1=((nitrate_df.iloc[:,1]*nitrate_df['molecular_cross_section']*nitrate_df['quantumn_yield']).sum(axis=0)*delta_lamda)
nitrate_2=((nitrate_df.iloc[:,2]*nitrate_df['molecular_cross_section']*nitrate_df['quantumn_yield']).sum(axis=0)*delta_lamda)
nitrate_3=((nitrate_df.iloc[:,3]*nitrate_df['molecular_cross_section']*nitrate_df['quantumn_yield']).sum(axis=0)*delta_lamda)
nitrate_4=((nitrate_df.iloc[:,4]*nitrate_df['molecular_cross_section']*nitrate_df['quantumn_yield']).sum(axis=0)*delta_lamda)
summary_df['nitrate']=[nitrate_1, nitrate_2, nitrate_3,nitrate_4]


In [ ]:
nitrite_df=(pd.merge(actinic, new_cross_section_nitrite, on='wavelength'))
delta_lamda= nitrite_df.wavelength[1]-nitrite_df.wavelength[0]
nitrite_1=((nitrite_df.iloc[:,1]*nitrite_df['molecular_cross_section']*nitrite_df['quantumn_yield']).sum(axis=0)*delta_lamda)
nitrite_2=((nitrite_df.iloc[:,2]*nitrite_df['molecular_cross_section']*nitrite_df['quantumn_yield']).sum(axis=0)*delta_lamda)
nitrite_3=((nitrite_df.iloc[:,3]*nitrite_df['molecular_cross_section']*nitrite_df['quantumn_yield']).sum(axis=0)*delta_lamda)
nitrite_4=((nitrite_df.iloc[:,4]*nitrite_df['molecular_cross_section']*nitrite_df['quantumn_yield']).sum(axis=0)*delta_lamda)
summary_df['nitrite']=[nitrite_1, nitrite_2, nitrite_3,nitrite_4]


In [ ]:
I2_df=(pd.merge(actinic, new_cross_section_I2, on='wavelength'))
delta_lamda= I2_df.wavelength[1]-I2_df.wavelength[0]
I2_1=((I2_df.iloc[:,1]*I2_df['molecular_cross_section']*I2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
I2_2=((I2_df.iloc[:,2]*I2_df['molecular_cross_section']*I2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
I2_3=((I2_df.iloc[:,3]*I2_df['molecular_cross_section']*I2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
I2_4=((I2_df.iloc[:,4]*I2_df['molecular_cross_section']*I2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
summary_df['I2']=[I2_1, I2_2, I2_3,I2_4]


In [ ]:
clno_df=(pd.merge(actinic, new_cross_section_clno, on='wavelength'))
delta_lamda= clno_df.wavelength[1]-clno_df.wavelength[0]
clno_1=((clno_df.iloc[:,1]*clno_df['molecular_cross_section']*clno_df['quantumn_yield']).sum(axis=0)*delta_lamda)
clno_2=((clno_df.iloc[:,2]*clno_df['molecular_cross_section']*clno_df['quantumn_yield']).sum(axis=0)*delta_lamda)
clno_3=((clno_df.iloc[:,3]*clno_df['molecular_cross_section']*clno_df['quantumn_yield']).sum(axis=0)*delta_lamda)
clno_4=((clno_df.iloc[:,4]*clno_df['molecular_cross_section']*clno_df['quantumn_yield']).sum(axis=0)*delta_lamda)
summary_df['clno']=[clno_1, clno_2, clno_3,clno_4]


In [ ]:
NO2_df=(pd.merge(actinic, new_cross_section_NO2, on='wavelength'))
delta_lamda= NO2_df.wavelength[1]-NO2_df.wavelength[0]
NO2_1=((NO2_df.iloc[:,1]*NO2_df['molecular_cross_section']*NO2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
NO2_2=((NO2_df.iloc[:,2]*NO2_df['molecular_cross_section']*NO2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
NO2_3=((NO2_df.iloc[:,3]*NO2_df['molecular_cross_section']*NO2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
NO2_4=((NO2_df.iloc[:,4]*NO2_df['molecular_cross_section']*NO2_df['quantumn_yield']).sum(axis=0)*delta_lamda)
summary_df['NO2']=[NO2_1, NO2_2, NO2_3,NO2_4]


In [29]:
NO2_chamber_df=(pd.merge(psi_chamber_df, new_cross_section_NO2, on='wavelength'))
# delta_lamda= NO2_chamber_df.wavelength[1]-NO2_chamber_df.wavelength[0]
# NO2_1=((NO2_chamber_df.iloc[:,1]*NO2_chamber_df['molecular_cross_section']*NO2_chamber_df['quantumn_yield']).sum(axis=0)*delta_lamda)

# NO2_chamber_df

In [ ]:
HONO_df=(pd.merge(actinic, new_cross_section_HONO, on='wavelength'))
delta_lamda= HONO_df.wavelength[1]-HONO_df.wavelength[0]
HONO_1=((HONO_df.iloc[:,1]*HONO_df['molecular_cross_section']*HONO_df['quantumn_yield']).sum(axis=0)*delta_lamda)
HONO_2=((HONO_df.iloc[:,2]*HONO_df['molecular_cross_section']*HONO_df['quantumn_yield']).sum(axis=0)*delta_lamda)
HONO_3=((HONO_df.iloc[:,3]*HONO_df['molecular_cross_section']*HONO_df['quantumn_yield']).sum(axis=0)*delta_lamda)
HONO_4=((HONO_df.iloc[:,4]*HONO_df['molecular_cross_section']*HONO_df['quantumn_yield']).sum(axis=0)*delta_lamda)
summary_df['HONO']=[HONO_1, HONO_2, HONO_3,HONO_4]


In [ ]:

book = load_workbook(excel_path)
writer = pd.ExcelWriter(excel_path, engine = 'openpyxl')
writer.book = book

new_cross_section_nitrate.to_excel(writer, sheet_name = 'nitrate')
new_cross_section_nitrite.to_excel(writer, sheet_name = 'nitrite')
new_cross_section_HONO.to_excel(writer, sheet_name = 'HONO')
new_cross_section_I2.to_excel(writer, sheet_name = 'I2')
new_cross_section_clno.to_excel(writer, sheet_name = 'clno')
summary_df.to_excel(writer,sheet_name='summary_photolysis_rate_constant')

writer.close()

/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_24045/644696310.py:3: FutureWarning:

Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version

/Users/cathysmacpro/miniconda3/envs/snowflakes/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning:

Title is more than 31 characters. Some applications may not be able to read the file

